# 요구사항1

### TitanicDataset 클래스

In [14]:
import torch
from torch.utils.data import Dataset

class TitanicDataset(Dataset):
    # 초기화 메서드로, 데이터셋의 특성(X)과 타겟(y)을 입력받아 변환
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)  # 입력 데이터를 FloatTensor로 변환
        self.y = torch.LongTensor(y)   # 타겟 데이터를 LongTensor로 변환

    # 데이터셋의 크기를 반환하는 메서드
    def __len__(self):
        return len(self.X)

    # 인덱스(idx)에 해당하는 데이터 포인트를 반환하는 메서드
    def __getitem__(self, idx):
        feature = self.X[idx]
        target = self.y[idx]
        return {'input': feature, 'target': target}

    # 객체 정보를 문자열로 반환하는 메서드
    def __str__(self):
        return f"Data Size: {len(self.X)}, Input Shape: {self.X.shape}, Target Shape: {self.y.shape}"


### TitanicTestDataset 클래스

In [15]:
class TitanicTestDataset(Dataset):
    # 초기화 메서드로, 테스트 데이터의 특성(X)을 입력받아 텐서로 변환
    def __init__(self, X):
        self.X = torch.FloatTensor(X)  # 테스트 데이터를 FloatTensor로 변환

    # 데이터셋의 크기를 반환하는 메서드
    def __len__(self):
        return len(self.X)

    # 인덱스(idx)에 해당하는 데이터 포인트를 반환하는 메서드
    def __getitem__(self, idx):
        feature = self.X[idx]
        return {'input': feature}

    # 객체 정보를 문자열로 반환하는 메서드
    def __str__(self):
        return f"Data Size: {len(self.X)}, Input Shape: {self.X.shape}"


In [16]:
import os
import pandas as pd
from torch.utils.data import random_split

def get_preprocessed_dataset():
    # 현재 파일의 경로를 가져와서 데이터 파일 경로를 설정
    CURRENT_FILE_PATH = os.path.dirname(os.path.abspath("__file__"))

    # 학습 및 테스트 데이터의 파일 경로 설정
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    # 학습 및 테스트 데이터 로드
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    # 전체 데이터프레임을 하나로 병합하여 전처리 수행
    all_df = pd.concat([train_df, test_df], sort=False)

    # 전처리 함수 호출
    all_df = get_preprocessed_dataset_1(all_df)
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)

    # 전처리된 데이터를 학습 및 테스트 데이터셋으로 분리
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # 학습 데이터셋을 학습 및 검증 데이터셋으로 분리
    dataset = TitanicDataset(train_X.values, train_y.values)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)

    return train_dataset, validation_dataset, test_dataset


### get_preprocessed_dataset_1

In [17]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df


### get_preprocessed_dataset_2

In [18]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df


### get_preprocessed_dataset_3

In [19]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 중앙값을 사용하여 Age 결측치 메우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean"]
    
    # 호칭별 중앙값을 원래 데이터프레임에 병합
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    
    # Age 결측치가 있는 경우 해당 호칭의 중앙값으로 채움
    all_df.loc[all_df["Age"].isnull(), "Age"] = all_df["honorific_age_mean"]
    
    # 임시로 사용한 중앙값 컬럼을 제거
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df


### get_preprocessed_dataset_4

In [20]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가 (부모/자식 + 형제/자매)
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 가족수가 0인 경우 혼자 탑승(alone) 여부를 나타내는 컬럼 생성
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)  # 결측치를 0으로 채움

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df


### get_preprocessed_dataset_5

In [21]:
def get_preprocessed_dataset_5(all_df):
    # 주요 honorific만 유지하고 나머지는 "other"로 대체
    all_df.loc[~(
        (all_df["honorific"] == "Mr") |
        (all_df["honorific"] == "Miss") |
        (all_df["honorific"] == "Mrs") |
        (all_df["honorific"] == "Master")
    ), "honorific"] = "other"
    
    # Embarked 컬럼의 결측치를 "missing"으로 채움
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df


### get_preprocessed_dataset_6

In [22]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 수치값으로 변환
    category_features = all_df.columns[all_df.dtypes == "object"]
    
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        
        # 범주형 데이터만 변환
        if all_df[category_feature].dtypes == "object":
            le = le.fit(all_df[category_feature])
            all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df


### MyModel

In [23]:
from torch import nn

class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()
    
    # 신경망 모델을 정의하는 부분
    self.model = nn.Sequential(
      nn.Linear(n_input, 30),  # 입력 차원(n_input)에서 30개의 노드로 연결되는 완전 연결 레이어
      nn.ReLU(),               # 비선형 활성화 함수 ReLU (Rectified Linear Unit)
      nn.Linear(30, 30),       # 30개의 노드에서 다시 30개의 노드로 연결
      nn.ReLU(),               # 비선형 활성화 함수 ReLU
      nn.Linear(30, n_output)  # 30개의 노드에서 최종 출력 차원(n_output)으로 연결
    )

  def forward(self, x):
    # 신경망을 통해 데이터를 전달하는 부분
    x = self.model(x)
    return x


### 테스트 함수

In [24]:
def test(test_data_loader):
  print("[TEST]")
  batch = next(iter(test_data_loader))  # 테스트 데이터 로더에서 첫 번째 배치를 가져옴
  print("{0}".format(batch['input'].shape))  # 입력 데이터의 형상을 출력
  
  # MyModel 초기화 (입력 크기: 11, 출력 크기: 2)
  my_model = MyModel(n_input=11, n_output=2)
  
  # 모델을 사용하여 입력 배치에 대한 예측 수행
  output_batch = my_model(batch['input'])
  
  # 각 샘플에 대해 가장 높은 확률을 가지는 클래스를 예측
  prediction_batch = torch.argmax(output_batch, dim=1)
  
  # 각 샘플의 인덱스와 예측된 클래스를 출력
  for idx, prediction in enumerate(prediction_batch, start=892):
      print(idx, prediction.item())


### 실행

In [25]:
from torch.utils.data import DataLoader

if __name__ == "__main__":
  # 데이터셋 로딩 및 전처리
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

  # 데이터셋의 크기를 출력
  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

  # 학습 데이터셋의 샘플을 출력
  for idx, sample in enumerate(train_dataset):
    print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

  print("#" * 50, 2)

  # DataLoader 생성 (학습, 검증, 테스트)
  train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  # 학습 데이터 로더의 배치 구조 출력
  print("[TRAIN]")
  for idx, batch in enumerate(train_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  # 검증 데이터 로더의 배치 구조 출력
  print("[VALIDATION]")
  for idx, batch in enumerate(validation_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("#" * 50, 3)

  # 테스트 함수 호출
  test(test_data_loader)


C:\Users\seongjae\AppData\Local\Temp\ipykernel_19120\1457447896.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)  # 결측치를 0으로 채움
C:\Users\seongjae\AppData\Local\Temp\ipykernel_19120\1834716123.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 2.0000,  1.0000, 29.0000,  0.0000,  0.0000,  0.0000,  2.0000, 21.1792,
         2.0000,  0.0000,  1.0000]): 0
1 - tensor([ 2.0000,  0.0000, 33.0000,  1.0000,  2.0000, 27.7500,  2.0000, 21.1792,
         3.0000,  3.0000,  0.0000]): 1
2 - tensor([ 1.0000,  1.0000, 42.0000,  0.0000,  0.0000, 26.2875,  2.0000, 87.5090,
         2.0000,  0.0000,  1.0000]): 1
3 - tensor([ 2.0000,  0.0000, 24.0000,  0.0000,  2.0000, 14.5000,  2.0000, 21.1792,
         3.0000,  2.0000,  0.0000]): 1
4 - tensor([ 3.0000,  1.0000, 30.0000,  0.0000,  0.0000,  8.0500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
5 - tensor([ 3.0000,  1.0000, 34.5000,  0.0000,  0.0000,  6.4375,  0.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
6 - tensor([  1.0000,   1.0000,   0.9200,   1.0000,   2.0000, 151.5500,   2.0000,
         87.5090,   0.0000,   3.0000,   0.0000]): 1
7 - te

# 요구사항2

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from datetime import datetime
import wandb
import argparse
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd


# 타이타닉 데이터셋 로드 및 전처리 함수
def load_and_preprocess_titanic_data():
    # CSV 파일로부터 타이타닉 데이터셋 로드
    data = pd.read_csv('train.csv')

    # Title 추출 및 Family Size 생성
    data['Title'] = data['Name'].apply(lambda name: name.split(',')[1].split('.')[0].strip())
    data['FamilySize'] = data['Parch'] + data['SibSp'] + 1

    # 불필요한 컬럼 제거
    data = data.drop(columns=['Name', 'Ticket', 'Cabin'])

    # 결측값 처리
    data['Age'].fillna(data['Age'].median(), inplace=True)
    data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)
    data['Fare'].fillna(data['Fare'].median(), inplace=True)

    # 범주형 데이터를 수치형으로 변환
    data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})
    label_encoder = LabelEncoder()
    data['Title'] = label_encoder.fit_transform(data['Title'])
    data = pd.get_dummies(data, columns=['Embarked'])

    # 특성(Features)과 타겟(Target) 분리
    X = data.drop(columns=['Survived']).values
    y = data['Survived'].values.reshape(-1, 1)

    # 데이터 정규화
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # PyTorch 텐서로 변환
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    return X, y


def get_data():
    X, y = load_and_preprocess_titanic_data()

    # 학습 및 검증 데이터셋 분할
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # PyTorch Dataset 생성
    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    validation_dataset = torch.utils.data.TensorDataset(X_val, y_val)

    # DataLoader 생성
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))

    return train_data_loader, validation_data_loader


class MyModel(nn.Module):
    def __init__(self, n_input, n_output, activation_function='ReLU', n_hidden_unit_list=None):
        super().__init__()

        # 기본값 설정
        if n_hidden_unit_list is None:
            n_hidden_unit_list = [20, 20]  # 기본 은닉층 크기

        # 활성화 함수 매핑
        activation_fn_map = {
            'ReLU': nn.ReLU(),
            'ELU': nn.ELU(),
            'LeakyReLU': nn.LeakyReLU(),
            'PReLU': nn.PReLU(),
        }
        activation_fn = activation_fn_map.get(activation_function, nn.ReLU())

        # 모델 구성
        self.model = nn.Sequential(
            nn.Linear(n_input, n_hidden_unit_list[0]),
            activation_fn,
            nn.Linear(n_hidden_unit_list[0], n_hidden_unit_list[1]),
            activation_fn,
            nn.Linear(n_hidden_unit_list[1], n_output),
        )

    def get_activation_function(self):
        if self.activation_function == 'ReLU':
            return nn.ReLU()
        elif self.activation_function == 'ELU':
            return nn.ELU()
        elif self.activation_function == 'LeakyReLU':
            return nn.LeakyReLU()
        elif self.activation_function == 'PReLU':
            return nn.PReLU()
        else:
            raise ValueError(f"Invalid activation function: {self.activation_function}")

    def forward(self, x):
        x = self.model(x)
        return x


def get_model_and_optimizer(activation_function):
    my_model = MyModel(n_input=12, n_output=1, activation_function=activation_function)
    optimizer = optim.Adam(my_model.parameters(), lr=wandb.config.learning_rate)

    return my_model, optimizer


def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.BCEWithLogitsLoss()  # 이진 분류에 적합한 손실 함수
    next_print_epoch = 100
    best_validation_loss = float('inf')
    patience = 10  # 조기 종료를 위한 기준 에포크 수
    trigger_times = 0

    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        num_trains = 0
        for train_batch in train_data_loader:
            input, target = train_batch
            output_train = model(input)
            loss = loss_fn(output_train, target)
            loss_train += loss.item()
            num_trains += 1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss_validation = 0.0
        num_validations = 0
        with torch.no_grad():
            for validation_batch in validation_data_loader:
                input, target = validation_batch
                output_validation = model(input)
                loss = loss_fn(output_validation, target)
                loss_validation += loss.item()
                num_validations += 1

        avg_validation_loss = loss_validation / num_validations
        wandb.log({
            "Epoch": epoch,
            "Training loss": loss_train / num_trains,
            "Validation loss": avg_validation_loss
        })

        # 조기 종료 조건 체크
        if avg_validation_loss < best_validation_loss:
            best_validation_loss = avg_validation_loss
            trigger_times = 0
            torch.save(model.state_dict(), f'best_model_{wandb.config.activation_function}.pth')  # 최적의 모델 저장
            print(f"Best model saved at epoch {epoch} with validation loss {best_validation_loss:.4f}")
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print(f"Early stopping triggered at epoch {epoch}")
                break

        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training loss {loss_train / num_trains:.4f}, "
                f"Validation loss {avg_validation_loss:.4f}"
            )
            next_print_epoch += 100


def main(args):
    activation_functions = ['ReLU', 'ELU', 'LeakyReLU', 'PReLU']

    for activation_function in activation_functions:
        current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

        config = {
            'epochs': args.epochs,
            'batch_size': args.batch_size,
            'learning_rate': 1e-3,
            'n_hidden_unit_list': [20, 20],
            'activation_function': activation_function  # Activation Function 설정
        }

        wandb.init(
            mode="online" if args.wandb else "disabled",
            project="titanic_model_training",
            notes=f"Titanic dataset experiment with {activation_function}",
            tags=["titanic", "classification", activation_function],
            name=f"{current_time_str}_{activation_function}",
            config=config
        )

        train_data_loader, validation_data_loader = get_data()
        linear_model, optimizer = get_model_and_optimizer(activation_function)

        print(f"Starting training with activation function: {activation_function}")

        training_loop(
            model=linear_model,
            optimizer=optimizer,
            train_data_loader=train_data_loader,
            validation_data_loader=validation_data_loader
        )
        wandb.finish()


if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--wandb", action=argparse.BooleanOptionalAction, default=False, help="True or False"
    )

    parser.add_argument(
        "-b", "--batch_size", type=int, default=512, help="Batch size (int, default: 512)"
    )

    parser.add_argument(
        "-e", "--epochs", type=int, default=1_000, help="Number of training epochs (int, default: 1_000)"
    )

    args = parser.parse_args()
    main(args)


# 요구사항 3

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from d_titanic_training import MyModel
from titanic_dataset import get_preprocessed_dataset


# 테스트 데이터 로드 및 예측 수행 함수
def generate_predictions(test_data_loader, model, device='cpu'):
    model.eval()  # 모델을 평가 모드로 전환
    predictions = []

    with torch.no_grad():
        for batch in test_data_loader:
            input_data = batch['input'].to(device)
            output = model(input_data)
            prediction_batch = torch.argmax(output, dim=1).cpu().numpy()
            predictions.extend(prediction_batch)

    return predictions


# submission.csv 파일 생성
def create_submission_file(predictions, output_file='submission.csv'):
    submission = pd.DataFrame({
        "PassengerId": range(892, 892 + len(predictions)),
        "Survived": predictions
    })
    submission.to_csv(output_file, index=False)
    print(f"Submission file '{output_file}' created successfully!")


if __name__ == "__main__":
    activation_function = 'ELU'
    n_hidden_unit_list = [20, 20]  # wandb 대신 기본값 설정

    # 모델 재구성
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = MyModel(n_input=11, n_output=2, activation_function=activation_function, n_hidden_unit_list=n_hidden_unit_list).to(device)

    # 훈련된 모델을 로드하거나 직접 훈련 후 모델 적용
    # model.load_state_dict(torch.load('best_model.pth'))  # 'best_model.pth'에 저장된 최적의 모델 로드

    # 테스트 데이터 로더 생성
    _, _, test_dataset = get_preprocessed_dataset()
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    # 예측 수행
    predictions = generate_predictions(test_data_loader, model, device=device)

    # submission.csv 파일 생성
    create_submission_file(predictions)


1. 테스트 수행 및 최적의 Epoch 선정
Activation Function:
요구사항 2에서 다양한 활성화 함수의 성능을 비교한 결과, 가장 좋은 성능을 보이는 Activation Function을 선정하였습니다.


Epoch 시점의 선택:
훈련 과정 중 매 Epoch마다 모델의 성능을 검증하여 Validation Loss가 가장 낮은 시점을 최적의 Epoch로 선정했습니다.
최적의 Epoch 시점에서 모델을 사용하여 최종 테스트를 수행하며 submission.csv를 생성하게 했습니다.
테스트 데이터는 test_data_loader를 활용하여 예측을 수행했습니다.


2. 추가 코드 구현
테스트 과정 중 최적의 Epoch를 판단할 수 있는 로직을 추가하였습니다.
모델의 Validation Loss가 개선될 때마다 최적의 모델을 저장하고, 이 모델을 이용하여 최종 테스트를 수행합니다.
submission.csv 파일을 생성하는 코드를 추가하였으며, 예측 결과를 기반으로 제출할 수 있는 형식으로 데이터를 정리하였습니다.

# 요구사항 4

s![](https://velog.velcdn.com/images/seongjae6751/post/00320357-a6c3-412d-bbf9-6d425cb934fe/image.png)

# 요구사항 5

![](https://velog.velcdn.com/images/seongjae6751/post/2f6d8278-48bd-4100-962c-91653e998c4f/image.png)


# 기술적 고찰 및 개선 사항

이번 과제를 통해 모델의 성능을 최적화하기 위해 다양한 Activation Function을 테스트하고 비교하는 경험을 했습니다. 이를 통해, 각 함수가 어떻게 모델의 학습 과정에 영향을 미치는지에 대해 깊이 있게 이해할 수 있었습니다.

특히, 조기 종료 기법을 사용하여 모델의 과적합을 방지하고 최적의 Epoch을 선택하는 것이 중요함을 깨달았습니다. 이를 바탕으로 Validation Loss를 기준으로 최적의 모델을 저장하고 활용함으로써 성능 향상을 도모할 수 있었습니다.

# 숙제 후기

이번 과제를 통해 다양한 Activation Function을 비교해보고, 최적의 모델을 선택하는 과정을 배우는 좋은 기회가 되었습니다. 특히, 모델 성능을 개선하기 위해 세부적인 조정을 하는 과정에서 딥러닝의 복잡성을 다시 한번 체감할 수 있었습니다.

개인적으로 가장 어려웠던 부분은 다양한 활성화 함수에 따른 성능 차이를 체계적으로 분석하고, 이에 맞춰 코드를 수정하는 과정이었습니다. 하지만, 이를 통해 얻은 경험은 앞으로 더 복잡한 모델을 다룰 때 큰 도움이 될 것이라 생각합니다. 이번 과제는 제가 딥러닝 모델의 최적화와 관련된 기술적 이해를 한 단계 더 높이는 기회가 되었습니다.